In [1]:
from magent2.environments import battle_v4, adversarial_pursuit_v4, tiger_deer_v4
from pettingzoo.utils import random_demo
import torch
import time
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from dqn_basic import DQN_Basic
from collections import namedtuple, deque
import numpy as np
# import matplotlib.pyplot as plt
from Animal import Animal
from pathlib import Path
from collections import deque
import datetime, os
from metric_logger import MetricLogger
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
frame_size = (128, 128)
max_cycles = 500
total_episodes = 1000
map_size = 40

# env = tiger_deer_v4.env(map_size=map_size, minimap_mode=False, render_mode='human', tiger_step_recover=-0.1, deer_attacked=-0.1, max_cycles=max_cycles, extra_features=False)
env = tiger_deer_v4.env(map_size=map_size, minimap_mode=False, render_mode='rgb_array', max_cycles=max_cycles, extra_features=False)

print(env.action_spaces['deer_0'].n)
print(env.observation_spaces['deer_0'].shape)
deer_observation_shape = env.observation_spaces['deer_0'].shape
deer_observation_shape = tuple(np.roll(deer_observation_shape,1))
deer_action_space = env.action_spaces['deer_0'].n

tiger_observation_shape = env.observation_spaces['tiger_0'].shape
tiger_observation_shape = tuple(np.roll(tiger_observation_shape,1))
tiger_action_space = env.action_spaces['tiger_0'].n

print(deer_observation_shape, tiger_observation_shape)
assert deer_observation_shape != None
assert tiger_observation_shape != None
# 'rgb_array' or 'human'
# random_demo(env, render=False , episodes=1)
# 'rgb_array'
# pygame.quit()

5
(3, 3, 5)
(5, 3, 3) (5, 9, 9)


c:\Users\mike\AppData\Local\Programs\Python\Python310\lib\site-packages\pettingzoo\utils\wrappers\base.py:64: UserWarning: The `action_spaces` dictionary is deprecated. Use the `action_space` function instead.
  warnings.warn(
c:\Users\mike\AppData\Local\Programs\Python\Python310\lib\site-packages\pettingzoo\utils\wrappers\base.py:50: UserWarning: The `observation_spaces` dictionary is deprecated. Use the `observation_space` function instead.
  warnings.warn(


In [3]:
def get_agent_type_from_agent_name(agent):
    if 'tiger' in agent:
        return 'tiger'
    return 'deer'

In [4]:
use_cuda = torch.cuda.is_available()
print(f"Using CUDA: {use_cuda}")
print()

deer_save_dir = Path("deer_checkpoints") / datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
deer_save_dir.mkdir(parents=True)

tiger_save_dir = Path("tiger_checkpoints") / datetime.datetime.now().strftime("%Y-%m-%dT%H-%M-%S")
tiger_save_dir.mkdir(parents=True)

deer = Animal(state_dim=deer_observation_shape, action_dim=deer_action_space, save_dir=deer_save_dir)
tiger = Animal(state_dim=tiger_observation_shape, action_dim=tiger_action_space, save_dir=tiger_save_dir)

deer_logger = MetricLogger(deer_save_dir)
tiger_logger = MetricLogger(tiger_save_dir)

activeDeer = True
activeTiger = True

for e in range(total_episodes):
    print("Episode: ", e)
    env.reset(seed=None)
    agents_prev_data = {}
    # Play the game!
    for agent in env.agent_iter():
        
        observation, reward, termination, truncation, info = env.last()
        observation = np.transpose(observation, (2,0,1))
        done = termination or truncation
        agentType = get_agent_type_from_agent_name(agent)
        # instantiate previous data
        if agent not in agents_prev_data.keys():
            if 'tiger' in agentType:
                action = np.random.randint(0,tiger_action_space,(1,))[0]
            else:
                action = np.random.randint(0,deer_action_space,(1,))[0]
            agents_prev_data[agent] = [observation, action, done, reward,  info]
            env.step(action)
            continue
        prev_observation = agents_prev_data[agent][0]
        prev_action = agents_prev_data[agent][1]
        prev_done = agents_prev_data[agent][2]
        
        if not prev_done:
            if 'deer' in agentType:
                if activeDeer:
                    deer.cache(prev_observation, observation, prev_action, reward, done, agentName=agent)
                    #learn
                    q, loss = deer.learn()
                    # logging
                    deer_logger.log_step(reward, loss, q)
                action = deer.act(observation)
            else:
                if activeTiger:
                    tiger.cache(prev_observation, observation, prev_action, reward, done, agentName=agent)
                    #learn
                    q, loss = tiger.learn()
                    # logging
                    tiger_logger.log_step(reward, loss, q)
                action = tiger.act(observation)
        
        if done:
            action = None
        agents_prev_data[agent] = [observation, action, done, reward,  info]
        env.step(action)
        

    deer_logger.log_episode()
    tiger_logger.log_episode()

    if e % 5 == 0:
        deer_logger.record(episode=e, epsilon=deer.exploration_rate, step=deer.curr_step)
        tiger_logger.record(episode=e, epsilon=tiger.exploration_rate, step=tiger.curr_step)
deer_logger.record(episode=e, epsilon=deer.exploration_rate, step=deer.curr_step)
tiger_logger.record(episode=e, epsilon=tiger.exploration_rate, step=tiger.curr_step)

Using CUDA: True

Episode:  0


c:\Users\mike\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


Animal saved to deer_checkpoints\2023-04-23T20-33-43\animal_net_0.chkpt at step 0
Animal saved to tiger_checkpoints\2023-04-23T20-33-43\animal_net_0.chkpt at step 0
Episode 0 - Step 8000 - Epsilon 0.9980019984175517 - Mean Reward -5.6 - Mean Length 8000.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 4.912 - Time 2023-04-23T20:33:48
Episode 0 - Step 1600 - Epsilon 0.9996000799392978 - Mean Reward 56.0 - Mean Length 1600.0 - Mean Loss 0.0 - Mean Q Value 0.0 - Time Delta 5.211 - Time 2023-04-23T20:33:48
Episode:  1


KeyboardInterrupt: 

<Figure size 432x288 with 0 Axes>

In [9]:
env = tiger_deer_v4.env(map_size=map_size, minimap_mode=False, render_mode='human', max_cycles=max_cycles, extra_features=False)
example_episodes = 10
for e in range(example_episodes):
    print("Episode: ", e)
    env.reset(seed=None)
    # Play the game!
    init_agent = None
    for agent in env.agent_iter():
        if init_agent is None:
            init_agent = agent
        else:
            if init_agent == agent:
                time.sleep(0.01)
        observation, reward, termination, truncation, info = env.last()
        observation = np.transpose(observation, (2,0,1))
        done = termination or truncation
        agentType = get_agent_type_from_agent_name(agent)
        if not done:
            if 'deer' in agentType:
                # deer.cache(prev_observation, observation, prev_action, reward, done, agentName=agent)
                action = deer.act(observation)
            else:
                # tiger.cache(prev_observation, observation, prev_action, reward, done, agentName=agent)
                action = tiger.act(observation)
        else:
            action = None
        env.step(action)
    

Episode:  0
Episode:  1
Episode:  2
Episode:  3
Episode:  4
Episode:  5
Episode:  6
Episode:  7
Episode:  8
Episode:  9
